# <center> Problem Set 4

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
from scipy.optimize import minimize
from scipy import stats
from scipy.optimize import fmin_bfgs
import math
import statsmodels.api as sm

df = pd.read_stata("PS4_data.dta")
#df.describe()

### 1. Select only male heads of household who are between 25 and 60 years of age and earn wages > $7/hr

In [2]:
#create wage variable using income and work hours
#df['wage'].describe()
df['wage2'] = df['hlabinc']/df['hannhrs']
df = df[(df.hsex==1)&(df.age>=25)&(df.age<=60)&(df.wage2>7)]

#create log wage variable
df['lnwage2'] = np.log(df['wage2'])
df = df[df.lnwage2!=np.inf]

#create age squared variable
df['agesq'] = df['age']**2

#df.agg({"age": ["mean", "min", "max"], "hsex": ["mean", "min", "max"]})
sumstats=df[['hsex','age','agesq','wage2','lnwage2']]
sumstats.describe().loc[['count','mean','max','min']]

,hsex,age,agesq,wage2,lnwage2
count,57477.0,57477.000000,57477.000000,57477.000000,57477.000000
mean,1.0,39.224247,1629.789673,24.306034,3.010414
max,1.0,60.000000,3600.000000,1717.330322,7.448526
min,1.0,25.000000,625.000000,7.000252,1.945946


### 2. Create indicator and continuous variables as necessary

In [3]:
#df['hrace'].describe()
df['White'] = (df['hrace'] == 1).astype(int)
df['Black'] = (df['hrace'] == 2).astype(int)
df['Hispanic'] = (df['hrace'] == 5).astype(int)
df['OtherRace'] = ((df['hrace']!=1)&(df['hrace']!=2)&(df['hrace']!=5)).astype(int)

#create df for the model
df = df[['lnwage2','hyrsed', 'age', 'agesq', 'Black', 'Hispanic', 'OtherRace', 'White', 'year']].dropna()

### 3. Estimate the following model via a Maximum Likelihood Estimator separately for t = 1971, 1980, 1990, 2000

#### Create Linear MLE Function

Nomral Log-Likelihood function:
\begin{equation}
\begin{gathered}
\ln \mathcal{L}(\theta \mid y, x)=-\frac{n}{2} \ln (2 \pi)-\frac{n}{2} \ln \left(\sigma^{2}\right)- \frac{1}{2 \sigma^{2}}(y-x \beta)^{\prime}(y-x \beta)
\end{gathered}
\end{equation}

In [4]:
class Linear_Model_MLE(object):
    '''
    This is a Class for Linear multiple regression model using MLE estimation
    The Optimaization method is BFGS, but you can modify the code and use other methods
    
    Function Linear_MLE: calculate the values for coefficent estimates, SE, variance covariance matrix, t-stats, and 
    p-value
    
    Function Summary: return regression results
                      For example: If you would like to see the regression summary table, 
                                   please write it like this example: Linear_Model_MLE(y,x, create_intercept=True).Summary()
    
    Inputs:
    y: dependent variable (Type: pd Series)
    x: independent variable (Type: pd DataFrame, or pd Series if there is only one x variable)
    create_intercept: True or False (Type: bool). The default is True. Alter based on your model. 
    '''
    
    def __init__(self, y, x, create_intercept=True):
        self.y = y
        self.x = x
        #self.y = self.y.to_numpy()
        #self.x = self.x.to_numpy()
        
        if create_intercept:
            self.x['intercept']=1
            #self.x = self.x.assign(intercept=pd.Series([1]*np.shape(self.x)[0]))
            self.x = self.x
        else:
            self.x = x

        if isinstance(create_intercept, bool):
            self.create_intercept = create_intercept
        else:
            raise RuntimeError("error")        

    def Linear_MLE(self):
        self.xt = self.x.T
        self.xx = self.xt @ (self.x)
        self.xxi = np.linalg.inv(self.xx)

        n = self.x.shape[0] #nrow
        k = self.x.shape[1] #ncol
        GuessVector = np.ones(k+1) # beta Vector is initial guess for beta value                
        
        def neglnL(theta):
            beta=theta[:-1]  #everything but the last item
            sigma=theta[-1:] #the last item
            yhat = self.x@beta
            
            lnL = -(n/2)*np.log(2*math.pi) - (n/2)*np.log(sigma**2) - (1/(2*sigma **2)) * (self.y-yhat)@(self.y-yhat).T
            #lnL = np.sum(stats.norm.logpdf(y, loc=yhat, scale=sigma)) #method 2
            return -lnL
          
        estimates = minimize(neglnL, GuessVector, method='BFGS')
        #estimates = fmin_bfgs(neglnL, x0=GuessVector, disp=False)
        
        bhat = estimates.x[:-1]
        var = (1/n) * (self.y- self.x@bhat) @ (self.y- self.x@bhat).T
        vcv = var * self.xxi
        standard_error = np.sqrt(np.diag(vcv))
        z_stat = bhat/standard_error
        p_value = t.sf(np.abs(z_stat), n-k)*2  

        self.results = {"Variable name": self.x.columns.values,
                        "coefficient": bhat, 
                        "standard_error": standard_error, 
                        "z_stat": z_stat, 
                        "p_value": p_value, 
                        "create_intercept": self.create_intercept}
     
    def summary(self):
        self.Linear_MLE()        
        print("labels","coefficient value","standard error","t-statistic","p-value",sep="\t    ")
        for i in range(len(self.results['Variable name'])):
            results11 = str(self.results['Variable name'][i]) + '\t' + '\t     ' + str(round(self.results['coefficient'][i], 4)) + '\t' + '\t' + '\t'  + str(round(self.results['standard_error'][i], 4)) + '\t              '  + str(round(self.results['z_stat'][i], 4)) + '\t    ' + str(round(self.results['p_value'][i], 4))
            print(results11)

### The model

$\ln \left(w_{i, t}\right)=\alpha+\beta_{1} E d u c_{i, t}+\beta_{2}$ Age $_{i, t}+\beta_{3} A g e_{i, t}^{2}+\beta_{4}$ Black $_{i, t}+\beta_{5}$ Hispanic $_{i, t}+\beta_{6}$ OtherRace $_{i, t}+\varepsilon_{i, t}$
w
where:
- $w_{i, t}=$ wage of individual $i$ in survey year $t$
- $E d u c_{i, t}=$ education in years
- $A g e_{i, t}=$ age in years
- Black$_{i, t}$, Hispanic$_{i, t}$, OtherRace$_{i, t}=$ dummy variables for race $=$ Black, Hispanic, Not $\in\{$ White, Black, Hispanic $\}$.

#### Estimate the model if t = 1971

In [5]:
df71 = df[df.year == 1971]
x = df71[['hyrsed', 'age', 'agesq', 'Black']]
y = df71['lnwage2']
Fit = Linear_Model_MLE(y,x, create_intercept=True)
Fit.summary()

<ipython-input-4-d90ba91b1ae0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x['intercept']=1


labels	    coefficient value	    standard error	    t-statistic	    p-value
hyrsed		     0.0665			0.0037	              17.9394	    0.0
age		     0.0647			0.0094	              6.8585	    0.0
agesq		     -0.0006			0.0001	              -5.3717	    0.0
Black		     -0.1647			0.0444	              -3.7069	    0.0002
intercept		     0.591			0.1929	              3.0638	    0.0022


In [6]:
#sm_OLS_fit = sm.OLS(endog=y, exog=x).fit()
#sm_OLS_results= sm_OLS_fit.summary()
#print(sm_OLS_results)

#### Estimate the model if t = 1980

In [7]:
df80 = df[df.year == 1980]
x = df80[['hyrsed', 'age', 'agesq', 'Black']]
#x = df80[['hyrsed', 'age', 'agesq', 'Black', 'Hispanic', 'OtherRace']]
y = df80['lnwage2']
Fit = Linear_Model_MLE(y,x, create_intercept=True)
Fit.summary()

<ipython-input-4-d90ba91b1ae0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x['intercept']=1


labels	    coefficient value	    standard error	    t-statistic	    p-value
hyrsed		     0.066			0.0043	              15.4827	    0.0
age		     0.0454			0.0096	              4.7505	    0.0
agesq		     -0.0004			0.0001	              -3.4405	    0.0006
Black		     -0.1033			0.0434	              -2.3814	    0.0173
intercept		     1.0064			0.1915	              5.2561	    0.0


In [8]:
#sm_OLS_fit = sm.OLS(endog=y, exog=x).fit()
#sm_OLS_results= sm_OLS_fit.summary()
#print(sm_OLS_results)

#### Estimate the model if t = 1990

In [9]:
df90 = df[df.year == 1990]
x = df90[['hyrsed', 'age', 'agesq', 'Black']]
#x = df90[['hyrsed', 'age', 'agesq', 'Black', 'Hispanic', 'OtherRace']]
y = df90['lnwage2']
Fit = Linear_Model_MLE(y,x, create_intercept=True)
Fit.summary()

<ipython-input-4-d90ba91b1ae0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x['intercept']=1


labels	    coefficient value	    standard error	    t-statistic	    p-value
hyrsed		     0.0955			0.0049	              19.5775	    0.0
age		     0.0577			0.0106	              5.4281	    0.0
agesq		     -0.0005			0.0001	              -4.1911	    0.0
Black		     -0.1672			0.0475	              -3.5201	    0.0004
intercept		     0.2799			0.2168	              1.2909	    0.1969


In [10]:
#sm_OLS_fit = sm.OLS(endog=y, exog=x).fit()
#sm_OLS_results= sm_OLS_fit.summary()
#print(sm_OLS_results)

#### Estimate the model if t = 2000

In [11]:
df2000 = df[df.year == 2000]
x = df2000[['hyrsed', 'age', 'agesq', 'Black']]
#x = df2000[['hyrsed', 'age', 'agesq', 'Black', 'Hispanic', 'OtherRace']]
y = df2000['lnwage2']
Fit = Linear_Model_MLE(y,x, create_intercept=True)
Fit.summary()

<ipython-input-4-d90ba91b1ae0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.x['intercept']=1


labels	    coefficient value	    standard error	    t-statistic	    p-value
hyrsed		     0.1103			0.0051	              21.6438	    0.0
age		     0.0841			0.0102	              8.2635	    0.0
agesq		     -0.0009			0.0001	              -7.2251	    0.0
Black		     -0.2578			0.0473	              -5.4504	    0.0
intercept		     -0.2908			0.2156	              -1.3488	    0.1775


#### Compare the MLE results against an StatsModel OLS estimator to confirm the results

In [12]:
sm_OLS_fit = sm.OLS(endog=y, exog=x).fit()
sm_OLS_results= sm_OLS_fit.summary()
print(sm_OLS_results)

                            OLS Regression Results                            
Dep. Variable:                lnwage2   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     184.9
Date:                Mon, 04 Oct 2021   Prob (F-statistic):          1.62e-139
Time:                        00:51:22   Log-Likelihood:                -2055.8
No. Observations:                2595   AIC:                             4122.
Df Residuals:                    2590   BIC:                             4151.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
hyrsed         0.1103      0.005     21.623      0.0

#### MLE vs OLS

My MLE model results are very similar to the StatsModel OLS results in terms of coefficents, SE, t-statisics, and p-value. I only showed the StatsModel OLS results for 2000. If you would like to see the StatsModel OLS result for other year, please uncomment the code provided, and run the code.

### 4. Interpret the coefficient β1. How do the returns to education change over time in these data?

I did not include Other_Race, Hispanic in the model because there are only a few observations. Although including Other_Race, Hispanic work in the OLS models, I used matrices in the MLE models. If I include those variables, it would have a singular matrix

The model suggests that all else equal, the positive effect of education on wages has increased over the years. In other words, the wage between the less educated people and highly educated people has increased. For example, the models suggest in 1971 and 1980, one more year of education lead to around a 0.066 dollar increase in wages. However, in 2000, the effect has increased to around 0.11 dollars. The wage variable is statistically significant in all models at a 1% level of significance. The wages in the model are deflated to 2005.